In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
    """
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:
    Human: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:    
    """
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
    """
)

pipeline_prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    pipeline_prompts=pipeline_prompts,
    final_prompt=final,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!",
        "question": "What is your favorite food?",
    }
)

Arrrrg! Me favorite food be a good ol' plate o' fish 'n' chips! The taste o' crispy battered fish and salty chips be enough to make me heart sing like a sea shanty! Arg arg!

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate o' fish 'n' chips! The taste o' crispy battered fish and salty chips be enough to make me heart sing like a sea shanty! Arg arg!")